# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [26]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [27]:
csv = "Output File/cleanoutput.csv"

start_df = pd.read_csv(csv)

start_df


,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed
0,avarua,CK,-21.21,-159.78,75.20,57,95,6.93
1,port alfred,ZA,-33.59,26.89,53.04,43,47,5.70
2,salalah,OM,17.02,54.09,80.60,94,75,3.36
3,fez,MA,34.04,-5.00,82.40,45,0,10.29
4,talnakh,RU,69.49,88.40,48.20,93,75,8.95
...,...,...,...,...,...,...,...,...
587,waingapu,ID,-9.66,120.26,78.19,69,62,14.00
588,riberalta,BO,-10.98,-66.10,79.97,63,81,1.92
589,badvel,IN,14.75,79.05,76.48,85,100,8.28
590,sladkovo,RU,55.53,70.34,66.94,75,0,3.89


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [28]:
start_df['Humidity'] = start_df['Humidity'].astype(float)
locations = start_df[["Latitude", "Longitude"]]
hum = start_df["Humidity"].astype(float)


fig = gmaps.figure()

heat_map = gmaps.heatmap_layer(locations, weights=hum,
                              dissipating=False, max_intensity=10,
                              point_radius=1)

fig.add_layer(heat_map)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [35]:
vacation_df = start_df

vacation_df1 = vacation_df[~(vacation_df['Wind Speed'] > 10)]
vacation_df2 = vacation_df1[~(vacation_df1['Cloudiness'] > 0)]
vacation_df3 = vacation_df2[~(vacation_df2['Max Temp'] > 85)]
vaca_final = vacation_df3[~(vacation_df3['Max Temp'] > 75)]
vaca_final.head()

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed
5,alta floresta,BR,-9.88,-56.09,72.16,41.0,0,2.66
24,kasama,ZM,-10.21,31.18,52.29,46.0,0,8.46
36,cape town,ZA,-33.93,18.42,43.00,87.0,0,1.12
44,arraias,BR,-12.93,-46.94,70.02,40.0,0,6.11
55,yulara,AU,-25.24,130.99,50.00,39.0,0,5.82


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [30]:
hotel_df = vaca_final
hotel_df.head()

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed
5,alta floresta,BR,-9.88,-56.09,72.16,41.0,0,2.66
24,kasama,ZM,-10.21,31.18,52.29,46.0,0,8.46
36,cape town,ZA,-33.93,18.42,43.00,87.0,0,1.12
44,arraias,BR,-12.93,-46.94,70.02,40.0,0,6.11
55,yulara,AU,-25.24,130.99,50.00,39.0,0,5.82


In [34]:
radious = 5000
tar_type = "lodging"

params = {
    "radius": radious,
    "type": tar_type,
    "key": g_key
}


for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    params["location"]=f"{lat},{lng}"
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(url, params=params)
    hotel_res = response.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"]=hotel_res["results"][0]["name"]
    except(KeyError, IndexError):
        print("Skip")
#print(json.dumps(hotel_res,indent=4,sort_keys=True))

hotel_df.head()
    

C:\Users\bradb\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip


,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
5,alta floresta,BR,-9.88,-56.09,72.16,41.0,0,2.66,Floresta Amazonica Hotel
24,kasama,ZM,-10.21,31.18,52.29,46.0,0,8.46,Kalebalika Cottages
36,cape town,ZA,-33.93,18.42,43.00,87.0,0,1.12,Southern Sun Waterfront Cape Town
44,arraias,BR,-12.93,-46.94,70.02,40.0,0,6.11,Hotel Japão
55,yulara,AU,-25.24,130.99,50.00,39.0,0,5.82,Desert Gardens Hotel - Ayers Rock Resort


In [33]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [37]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))